This notebook helps with remove solvent/ion from an ionic MOFs. Below are the steps we did to remove solvent and check the chemical formula.

1. Construct a neighborlist using ASE neighborlist.
2. Construct a graph representation of the MOF from the neighborlist, using atoms as nodes and bonds (determined from neighborlist) as edges.
3. Generate the connected components (determine subgraphs) of the graph representation.
4. Determine the largest component by mass.
5. Compare the chemical formula of (a) largest fragment of CIF (from step (4)) and (b) data from CSD. If (1) the elements match between (a) and (b), and (2) for each element, the ratio between the number of that element in (a) and (b) is the same, then keep the MOF, and go to step 6. Else return False and continue
6. Removal subgraphs that have a mass less than 0.8* mass of the largest component. The large cutoff is due to the possibility of partial occupancy of solvent/ion.
7. Write the CIF after solvent removal.

In [18]:
from MOFChemicalFormula import MOFChemicalFormula
from ase.io import read as ase_read
from ase.io import write as ase_write
from ase import neighborlist
from ase.formula import Formula
from ase.build import sort

import networkx as nx
import glob
import subprocess
import json

def remove_solvent(cif, output_directory, formula, added_identifier='_removed_ions'):
    """ Remove solvent based on connected components and ASE neighbor dictionary. TODO: Consider formula

    Parameters:
        cif: str, path_to_cif file to remove solvent
        output_directory: str, path to write solvent removed CIF.
        formula: str, chemical formula of the MOFs framework. 
                 Use to check if after removing solvents, any additional atoms are incorrectly removed. Default: None
    Return:
        check: bool, if the chemical formula between 
    """

    def dict2str(dct):
        """Convert symbol-to-number dict to str.
        """
        return ''.join(symb + (str(n)) for symb, n in dct.items())

    try:    
        mof = ase_read(cif)
    except:
        print("Cannot read {} using ASE".format(cif))
        return False, 'Error'
    
    if '/' in cif:
        name = cif.split('/')[-1].split('.')[:-1] # If '/' in path to CIF
    else:
        name = cif.split('.')[:-1] # CIF file name 
    if len(name) == 1:
        output_name = name[0] + added_identifier + '.cif'
    else:
        output_name = '.'.join(name) + added_identifier + '.cif'

    # 1. Construct a neighborlist using ASE neighborlist.

    cutOff = neighborlist.natural_cutoffs(mof)
    neighborList = neighborlist.NeighborList(cutOff, self_interaction=False, bothways=True) 
    neighborList.update(mof)

    #2. Construct a graph representation of the MOF from the neighborlist, using atoms as nodes and bonds 
    #   (determined from neighborlist) as edges.
    G = nx.Graph()

    for k in range(len(mof)):
        tup = (k, {"element":"{}".format(mof.get_chemical_symbols()[k]), "pos": mof.get_positions()[k]})
        G.add_nodes_from([tup])

    for k in range(len(mof)):
        for i in neighborList.get_neighbors(k)[0]:
            G.add_edge(k, i)

    # 3. Generate the connected components (determine subgraphs) of the graph representation.
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    
    ions_index = []
    formulas = []
    input_formula = MOFChemicalFormula(formula)
    
    # 4. Determine the largest component by mass.
    if len(Gcc) > 1:
        massG = []
        for index, g in enumerate(Gcc):
            g = list(g)
            fragment = mof[g]
            fragment = sort(fragment)
            massG.append(sum(mof[g].get_masses())) # Mass of each disconnected subgraph
            form_dict = fragment.symbols.formula.count()
            formulas.append(dict2str(form_dict))
            
        cform = MOFChemicalFormula(','.join(formulas))
        cform_largest_fragment = MOFChemicalFormula(cform.get_largest_fragment())
        clf_element_dict = cform_largest_fragment.element_dict()
        
        """
        5. Compare the chemical formula of (a) largest fragment of CIF (from step (4)) and (b) data from CSD. 
        If (1) the elements match between (a) and (b), and (2) for each element, 
        the ratio between the number of that element in (a) and (b) is the same, then keep the MOF, 
        and go to step 6. Else return False and continue
        """
        check, multiplier = input_formula.is_multiplier(clf_element_dict)

        if check == True:
            lowerM = massG[1:]
            for index, mass in enumerate(lowerM):
                if mass/massG[0] < 0.8:
                    for n in Gcc[index+1]:
                        ions_index.append(n)

            ions_index = sorted(ions_index, reverse=True)
            del mof[ions_index]
            ase_write('{}/{}'.format(output_directory, output_name), mof)
            return check, multiplier

        elif check == False:
            print("Solvent removal failed for {}".format(cif))
            print(f"Input dictionary from CSD: {dict(input_formula.element_dict())}")
            print(f"Dictionary of largest fragment from CIF: {dict(clf_element_dict)}")
            return check, None
    
    elif len(Gcc) == 1:
        form_dict = mof.symbols.formula.count()
        cformula = dict2str(form_dict) 
        cform = MOFChemicalFormula(cformula)
        cform_largest_fragment = MOFChemicalFormula(cform.get_largest_fragment())
        clf_element_dict = cform_largest_fragment.element_dict()
        check, multiplier = input_formula.is_multiplier(clf_element_dict)
        if check ==  True:
            ase_write('{}/{}'.format(output_directory, output_name), mof)
            return check, multiplier
        else:
            return check, None


Because there are more than 14,000 ionic MOFs, we only show some examples of solvent removal. Crystal structure of MOFs after solvent removal are included in .tar.gz file

In [19]:
# Load data

f = open('example_cifs/ionic_MOFs_formula_example.json', 'r')
data = json.load(f)
f.close()

cifpath = 'example_cifs/before_solvent_removal'
output_dir = 'example_cifs/after_solvent_removal'

with open('log.txt', 'w') as f:
    for mof in data:
        formula = data[mof]['framework'][0]['formula']
        check, multiplier = remove_solvent(cifpath + '/' + mof + '.Non-disordered_MOF_subset.cif', output_dir, formula)
        print(f"Refcode: {mof}, Solvent removed: {check}, Number of formula unit in the CIF: {multiplier}")
        if check == True:
            f.write('{},{}\n'.format(mof, multiplier))
        else:
            f.write('{},FAIL\n'.format(mof))

Refcode: ABAYIO, Solvent removed: True, Number of formula unit in the CIF: 16
Refcode: FIQDAN, Solvent removed: True, Number of formula unit in the CIF: 2
Solvent removal failed for example_cifs/before_solvent_removal/ADOFOO.Non-disordered_MOF_subset.cif
Input dictionary from CSD: {'C': 16, 'H': 32, 'N': 4, 'Ni': 1, 'O': 4}
Dictionary of largest fragment from CIF: {'C': 32, 'H': 48, 'N': 8, 'Ni': 2, 'O': 8}
Refcode: ADOFOO, Solvent removed: False, Number of formula unit in the CIF: None


The solvent removal script works for ABAYIO and FIQDAN, but fails for ADOFOO. By manually examining of the refcode ADOFOO, we found that there are bound water on the Ni. However, in the CIF file, H atoms of those bound water are missing.

The output 'log.txt' documents the multiplier (in case the solvent removal works correct), or "FAIL" if it does not. The net charge of the CIF can be obtained by taking the product of this multiplier and the net charge of the formula (obtained from CSD) for each ionic MOF.

If one doesn't have access to the chemical formula, or don't worry about missing atoms, the script can be easily modified to just remove solvent.